<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Eval4pptx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands on: Eval - small LLM as a judge

Goal
* see how llm-as-a-judge works in principle
* introduction to the G-Eval algorithm ([G-Eval on arxive ](https://arxiv.org/abs/2303.16634))
* see how the algorithm uses prompts to generate the actual eval prompt
* try out the [DeepEval library](https://docs.confident-ai.com/docs/guides-using-custom-llms)
* see the limitations of small llms as a judge
* optional: compare to Gpt-4o

# SetUp : create an *llm_run* method using a small LLM

* load & quantize a small model from huggingface
* define a simple **llm_run** method, that calls the loaded model
* try out llm_run

=> same setup as in Assement notebook

In [1]:
!nvidia-smi

Wed Aug 28 15:36:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### **Important:**
Ensure that no GPU memory is allocated yet (in the case of a T4 look for "0MiB / 15360MiB").
If GPU memory is already allocated use Runtime/Manage Sessions to delete all active sessions.

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
%%time

!pip install --upgrade -q transformers accelerate flash_attn torch bitsandbytes
!pip install lm-format-enforcer -q
!pip install deepeval==1.1.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 35.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.9/365.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29

#### => you may need to restart the session

In [4]:
from google.colab import userdata

# Configure HuggingFace token as a Colab Secret, use key symbol on the left panel
!huggingface-cli login --token {userdata.get('HF_TOKEN')}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Load & quantize Model (yielding: model_id, model, tokenizer)

In [5]:
# kind = 'Lllama_3.1_8B_4bit'
kind = 'Lllama_3.1_8B_8bit'
# kind = 'Lllama_3.1_8B_16bit'  # too large for T4
# kind = 'Phi-3.5-MoE_4bit'     # No module named 'triton' ???
# kind = "Phi-3.5-mini_16bit"   # not "strong" enough

if "Lllama_3.1_8B" in kind:
  model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
elif "Phi-3.5-MoE" in kind:
  model_id = "microsoft/Phi-3.5-MoE-instruct"
else:
  model_id = "microsoft/Phi-3.5-mini-instruct"

print(kind)
print(model_id)

Lllama_3.1_8B_8bit
meta-llama/Meta-Llama-3.1-8B-Instruct


***note:*** execute in a terminal 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [6]:
%%time

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

torch_dtype = None
quantization_config = None

if "8bit" in kind:
  print("Using 8Bit quantization")
  quantization_config = BitsAndBytesConfig(load_in_8bit=True)
elif "4bit" in kind:
  print("Using 4Bit quantization")
  quantization_config = BitsAndBytesConfig(load_in_4bit=True)
else:
  print("Using Full Resolution")
  torch_dtype = torch.bfloat16

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    torch_dtype=torch_dtype,
    device_map="cuda",
    trust_remote_code=True
)

Using 8Bit quantization


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

CPU times: user 41.6 s, sys: 50.3 s, total: 1min 31s
Wall time: 4min 5s


In [7]:
!nvidia-smi

Wed Aug 28 15:41:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0              28W /  70W |   8825MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
from transformers import AutoTokenizer

def llm_run(messages):
  if type(messages) == str:
    messages = [{"role": "user", "content": messages}]
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  input_token_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
  ).to(model.device)

  outputs = model.generate(
      input_token_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=False
  )
  output_token_ids = outputs[0][input_token_ids.shape[-1]:]
  result = tokenizer.decode(output_token_ids, skip_special_tokens=True)
  return result

Try out our model:

In [9]:
%%time
print(model_id)
llm_run("who are you ?")

meta-llama/Meta-Llama-3.1-8B-Instruct


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: user 11.6 s, sys: 753 ms, total: 12.3 s
Wall time: 20 s


'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'

In [10]:
from IPython.display import Markdown

messages = [
    {"role": "system", "content": "You are an English-speaking, competent expert in the field of sanitary piping systems.."},
    {"role": "user", "content": f'''What are waste-water pipes made out of ?'''}
  ]

#answer = llm_run(messages)
#Markdown(answer)

# LLM-as-a-judge: in principle

In [11]:
llm_output="Witing texts is painful, caus im making mitakes."

In [12]:
simple_eval_prompt = f'''
You are an expert on english language, grading a students text with scores between 0 and 10.
A text written in proper english, in a fluent style, containing no grammatical or syntax errors is graded 10.
A text written in a different language or with spelling errors gets a low score.
Also give a detailed explanation why the score was chosen.
Do not repeat the students text in your explanation.

Always answer in the following json format:
{{
    "score": 8,
    "reason": "some reason"
}}

Examples
1. Student Text: Pipes are cylindrical conduits used to transport fluids or gases, typically made of materials like metal, plastic, or concrete.
   Answer:
   {{
    "score": 8,
    "reason": "The text is written in english and does not contain any syntactical or grammatical erros"
  }}
2. Student Text: Zwischen Neonlichtern und Straßenlärm träum ich leise von Freiheit.
   Answer:
   {{
    "score": 2,
    "reason": "The text is written in german and not in english."
  }}

Student Text: {llm_output}
Answer:
'''

In [13]:
%%time
import json

answer=llm_run(simple_eval_prompt)

json.loads(answer)

CPU times: user 16.7 s, sys: 63.1 ms, total: 16.8 s
Wall time: 17.5 s


{'score': 1,
 'reason': "The text contains multiple spelling errors, such as 'Witing' instead of 'Writing', 'caus' instead of 'because', and'mitakes' instead of'mistakes'. Additionally, the text is written in a non-standard dialect of English, which affects its clarity and coherence."}

# Llm-as-a-judge: G-Eval in principal

### G-Eval Step 1: generate evaluation steps based on the criteria

In [14]:
llm_input="Why do you dislike writing texts ?"
llm_output="Witing texts is painful, caus im making mitakes."

In [15]:
%%time
criteria="Grade the english grammar and syntax"

geval_step1_prompt = f'''Given an evaluation criteria which outlines how you should judge the Actual Output, generate
3-4 concise evaluation steps based on the criteria below. You MUST make it clear how to evaluate Actual Output in
relation to one another.

Evaluation Criteria:
{criteria}

**
IMPORTANT: Please make sure to only return in JSON format, with the "steps" key as a list of strings. No words or
explanation is needed.
Example JSON:
{{
    "steps": <list_of_strings>
}}
**

JSON:
'''

answer1 = llm_run(geval_step1_prompt)
json_answer1=json.loads(answer)
json_answer1


CPU times: user 20.2 s, sys: 0 ns, total: 20.2 s
Wall time: 22.2 s


{'steps': ['Check for correct grammar and punctuation usage in the Actual Output.',
  'Compare the Actual Output to the expected English language rules for verb tense, subject-verb agreement, and sentence structure.',
  'Evaluate the Actual Output for correct spelling and vocabulary usage.',
  'Assess the coherence and logical flow of the Actual Output in relation to the input or context.']}

In [16]:
steps="\n".join(f"{index+1}. {step}" for index, step in enumerate(json_answer1['steps']))
print(steps)

1. Check for correct grammar and punctuation usage in the Actual Output.
2. Compare the Actual Output to the expected English language rules for verb tense, subject-verb agreement, and sentence structure.
3. Evaluate the Actual Output for correct spelling and vocabulary usage.
4. Assess the coherence and logical flow of the Actual Output in relation to the input or context.


### G-Eval Step 2: evaluate the llm_output using the generated steps

In [17]:
geval_step2_prompt = f'''
Given the evaluation steps, return a JSON with two keys:
1) a `score` key ranging from 0 - 10, with 10 being that it follows the criteria outlined in the steps and 0 being that it does not, and
2) a `reason` key, a reason for the given score, but DO NOT QUOTE THE SCORE in your reason.
Please mention specific information from Actual Output and Input in your reason, but be very concise with it!

Evaluation Steps:
{steps}

Actual Output:
{llm_output}

Input:
{llm_input}



**
IMPORTANT: Please make sure to only return in JSON format, with the "score" and "reason" key. No words or explanation is needed.

Example JSON:
{{
    "score": 0,
    "reason": "The text does not follow the evaluation steps provided."
}}
**

JSON:

'''
print(geval_step2_prompt)

NameError: name 'actual_output' is not defined

In [ ]:
answer2=llm_run(geval_step2_prompt)
json.loads(answer2)

# G-Eval Implementation by DeepEval

A little adapter to connect our **llm_run** function with the deepEval library (and do some logging)

see: https://docs.confident-ai.com/docs/guides-using-custom-llms

In [ ]:
from deepeval.models import DeepEvalBaseLLM

log_output=""
llm_log = True

def log(log_message):
    global log_output

    if llm_log:
          log_output += log_message + "\n"

# wrapper calling llm_run using the global members model_id, model
class CustomDeepEvalLlm(DeepEvalBaseLLM):
    def __init__(self):
        super().__init__()
        self.generate_count = 0

    def load_model(self):
        return model

    def generate(self, prompt: str) -> str:
        self.generate_count += 1
        count = self.generate_count
        log(f'[{count}] ********************** deepEval LLM Generate BEGIN ********************************************')
        log(f'[{count}] ***** Prompt         : ' + prompt)

        result = llm_run(prompt)

        log(f'[{count}] ***** Answer         : ' + result)
        log(f'[{count}] ********************** deepEval LLM Generate END   ')
        log(f'[{count}] ')
        return result

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return model_id

deepeval_custom_model = CustomDeepEvalLlm()

try out the wrapper and check log output

In [ ]:
deepeval_custom_model.generate('who are you ?')

In [ ]:
print(log_output)

**geval_run** given calls the deepEval's geval implementation passing our criteria, llm_input and llm_output

In [ ]:
import deepeval
import deepeval.metrics
import deepeval.test_case
from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

def geval_run(name, criteria, input, output):
    test_case = deepeval.test_case.LLMTestCase(
        input=input,
        actual_output=output
      )

    metric = GEval(
        name=name,
        criteria=criteria,
        evaluation_params=[
            LLMTestCaseParams.ACTUAL_OUTPUT,
            LLMTestCaseParams.INPUT],
        model=deepeval_custom_model
    )

    eval_result = deepeval.evaluate(
        test_cases=[test_case],
        metrics=[metric]
    )
    return eval_result

In [ ]:
llm_input="What is a pipe ?"
llm_output_concise="A pipe is a tubular conduit used to transport fluids or sometimes solids. Pipes are typically made of materials like metal, plastic, or concrete."
llm_output_inconcise="Pipes are beautiful, black and round. Because they are round they are very convenient and don't have any edges."

In [ ]:
%%time
log_output=""
r = geval_run("Conciseness",
              "Determine how concise the actual output is.",
              llm_input,
              llm_output_inconcise )

In [ ]:
def print_metrics_data(deep_eval_result):
    for testcase in deep_eval_result:
      print("input        :", testcase.input)
      print("actual_output:", testcase.actual_output)
      for metric in testcase.metrics_data:
        print("name         :",metric.name)
        print("score        :",metric.score)
        print("reason       :",metric.reason)
        print("model        :",metric.evaluation_model)
        print()
      print("-----------")

In [ ]:
print_metrics_data(r)
r

In [ ]:
print(log_output)

In [ ]:
%%time
log_output=""
llm_input="What is a pipe ?"
llm_output_concise="A pipe is a tubular conduit used to transport fluids or sometimes solids. Pipes are typically made of materials like metal, plastic, or concrete."
llm_output_inconcise="Pipes are beautiful, black and round. Because they are round they are very convenient and don't have any edges."

r = geval_run("Conciseness", "Determine how concise the actual output is", llm_input, llm_output_inconcise )

In [ ]:
print(log_output)

# Evaluating multiple metrics: "Conciseness", AnswerRelevance, Toxicity,...

check out some other metrics [https://docs.confident-ai.com/docs/metrics-introduction](https://docs.confident-ai.com/docs/metrics-introduction)

In [ ]:
deepeval_custom_model = CustomDeepEvalLlm()

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric, ToxicityMetric

def metrics_run(input, output):
    test_case = deepeval.test_case.LLMTestCase(
        input=input,
        actual_output=output
      )

    conciseness_metric = GEval(
        name="Conciseness",
        criteria="Determine how concise the actual output is",
        evaluation_params=[
            LLMTestCaseParams.ACTUAL_OUTPUT,
            LLMTestCaseParams.INPUT],
        model=deepeval_custom_model
    )

    metrics = [
        conciseness_metric,
        AnswerRelevancyMetric(model=deepeval_custom_model),
       # ToxicityMetric(model=deepeval_custom_model) # Lllama_3.1_8B_8bit not "strong" enough
    ]

    eval_result = deepeval.evaluate(
        test_cases=[test_case],
        metrics=metrics,
    )
    return eval_result

In [ ]:
log_output=""
metrics_run(llm_input, llm_output)

In [ ]:
print(log_output)


# Optional: Compare with OpenAI gpt-4o

To use OpenAI you need an Api key. deepEval defaults to openAI if no model is set.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
deepeval_custom_model = None

In [ ]:
log_output=""
r=metrics_run(llm_input, llm_output_inconcise)

In [ ]:
print_metrics_data(r)